In [1]:
import numpy as np

## Recreating Zhou and Li protocol by generating the necessary parameter

In [2]:

n = int(input('size of matrix: ')) #Since the input is a square matrix 


R = np.random.rand(n, n)
A = R.T @ R #This creates a randomly generated square symmetric matrix. (This is the input) 
# R.T is the transpose of R.



size of matrix: 100


## Encrypting

In [3]:
alpha = np.random.rand()
s = np.random.rand()

P = np.eye(n)[:, np.random.permutation(n)] #Random permutation matrix
signs = np.random.choice([-1, 1], size=n)
M = P * signs #Creates a permutation matrix with non-zero entries as 1 and -1 (as per Zhou and Li(2016) protocol)

# Encryption
B = M @ (alpha*A + s*np.eye(n)) @ M.T

## Cheating performed by cloud

In [4]:
# Eigendecomposition
D, Y = np.linalg.eig(B) 
D1 = np.diag(D) #Formatting D as a 2-D array (n*n)

#Cheating

# Generating random 'cheating' vector k
k = np.random.randn(Y.shape[0],1).reshape(-1) # Randomly generated n*1 vector
k1 = np.square(k) #n*1 vector where k1[i] = k[i]^2

# Distorting the eigendecomposition
D_prime = np.zeros_like(D)
for i in range(len(D)):
    D_prime[i] = D[i] / k1[i]  #Dividing i-th eigenvalue by k[i]^2 
Lambda_prime = np.diag(D_prime)

Y_prime = np.zeros_like(Y)
for i in range(len(k)):
    Y_prime[:, i] = Y[:, i] * k[i] #Multiplying i-th eigenvector with k[i]

## Passing undetected in verification

In [5]:
def is_diagonal_matrix(Z):
    """
    Check if the given matrix Z is a diagonal matrix.
    """
    return np.all(Z == np.diag(np.diagonal(Z)))

def check(Z):
    """
    Check if Z is not a diagonal matrix and output a message if it is not.
    """
    if not is_diagonal_matrix(Z):
        print("reject the wrong result")
        return
    # If Z is a diagonal matrix, you can add any other code here if needed.
    print("It is a diagonal matrix.")

check(Lambda_prime)

It is a diagonal matrix.


In [6]:
l=int(input('No. of trials= ')) #For verification checks

for i in range(l):
    e = np.random.choice([0, 1], size=(Y_prime.shape[0]))
    err = np.linalg.norm(Y_prime @ (Lambda_prime @ (Y_prime.T @ e)) - B @ e)

    if err >= 1e-4:
        print("Verification failed")
        break

else:
    print("Verification successful for all trials")

No. of trials= 80
Verification successful for all trials


In [7]:
np.linalg.norm(Y_prime @ (Lambda_prime @ (Y_prime.T))- B)  
#For assurance (Representing both quantity are computationally equal)

9.203930654292488e-12

## Highlighting the mathematical issue. If Y_prime* Lambda_prime* Y_prime.T is indeed the eigendecomposition of B, then B* Y_prime should be equal to Y_prime* Lambda_prime.

In [8]:
def is_equal(A,B):
    """
    Check if the given matrices A and B are equal.
    """
    return np.array_equal(A, B)

is_equal(B@Y_prime,Y_prime@Lambda_prime)

False

## This shows that the malicious cloud has successfully provided incorrect results that passes the verification process.
## This also shows that i-th column of Y_prime i.e Y_prime[:,i] is NOT the eigenvector corresponding to the i-th eigenvalue in Lambda_prime. 

In [9]:
#Confirming the above statement (checking only the first column, can be replaced with any column within n)

is_equal(B@Y_prime[:,0],D_prime[0]*Y_prime[:,0])

False

## Moreover, as the cheated results passes the verification step, the client goes for decryption to retrieve the eigendecomposition of A from the incorrect eigendecomposition of B

In [10]:
#Decryption

X = M.T @ Y_prime 
L = Lambda_prime - s*np.eye(n)
Lambda  = (1/alpha) * L

# But the decryption proposed by Zhou and Li will not produce the eigendecomposition of A  

In [11]:
is_equal(A,X@Lambda@X.T)

False

In [12]:
l=int(input('No. of trials= ')) #For solid proof

for i in range(l):
    e1 = np.random.choice([0, 1], size=(X.shape[0]))
    err1 = np.linalg.norm(X @ (Lambda @ (X.T @ e1)) - A @ e1)

    if err1 >= 1e-4:
        print("Not Equal")
        break

else:
    print("Equal")

No. of trials= 1
Not Equal


In [13]:
np.linalg.norm(X @ (Lambda @ (X.T))-A) 
#Hard evidence that both matrices are not equal

8.382073560047575

# This shows that the cheated results has passed the verification process providing cheated eigendecomposition of B, which resulted in retrieving incorrect eigendecomposition of A.

## Proposed Verification Scheme (To show the effectiveness of the correction)

In [14]:
l=int(input('No. of trials'))
for i in range(l):
    e2 = np.random.choice([0, 1], size=(Y_prime.shape[0]))
    err2 = np.linalg.norm(Y_prime.T @ (Y_prime @ e2) - e2)
    
    if err2 >= 1e-4:
        print("Verification failed")
        break
    
    
    e1 = np.random.choice([0, 1], size=(Y_prime.shape[0]))
    err1 = np.linalg.norm(Y_prime @ (Lambda_prime @ (Y_prime.T @ e1)) - B @ e1)
    
    if err1 >= 1e-4:
        print("Verification failed")
        break
else:
    print("Verification successful for all trials")

No. of trials1
Verification failed


## Proposed verification scheme can detect the cheated outputs in one trial.